# Einstein Toolkit tutorial
<br/><br/><br/>
<center>
Roland Haas, NCSA and UIUC

rhaas@ncsa.illinois.edu
</center>

In [2]:
# let me use magic commands without the % sign
%automagic on


Automagic is ON, % prefix IS NOT needed for line magics.


In [3]:
%%HTML
<style>
.rendered_html ul { display: block; }
.floatright { float: right; padding-left: 1em; }
</style>

TODO: add intro slide with convetions used in this notebook

Topics to cover:

  1. a generic introduction to the Einstein Toolkit and how to use it. Similar to the presentation that Steve Brandt and Peter Diener gave at the NCSA ET workshop. See the program (has link to the recording) http://www.ncsa.illinois.edu/Conferences/ETK17/program.html and https://drive.google.com/drive/folders/0B4gNfWainf-5aGNiZ1IyUk9ZN2s .
  1. an introduction on how to set up a simulation campaign and how to use the Einstein Toolkit to simulate binary black hole or binary neutron stars.


# Structure of the toolkit
  * Collaborative, distributed code development <img style="float:right;width:20em" src="fig/logos.png"/>
  * Collaborations among competitors
  * Infrastructure modules
    * Parameter file handling
    * Parallelization (MPI)
    * Mesh refinement
    * IO, checkpointing
  * Physics modules
    * Spacetime evolution
    * Ideal general relativistic magnetohydrodynamics
    * Multigrid elliptic solver
  * Third party modules
    * Curvilinear grid coordinates (Llama)
    * Neutrino radiation transport codes (ZelmaniLeak, ZelmaniM1)
    * Data analysis tools (SimulationTools)

# Goals of this tutorial

Cactus is a very large code that has grown quite complex over the 20 years of its existence. This tutorial attempts to provide you with enough knowledge experience to:

  1. understand key Cactus concepts
  2. run a Cactus simulation (campaign) to study for example gravitational waves
  3. extract information about gravitational waves from the simulation
  <img style='float: right; height: 6ex' src="fig/kranc.png"/>
  4. modify existing Cactus modules (maybe)
    - using `C` and `Fortran` code
    - using `Kranc`
    
There will be plenty of hands on exercises that help understand the concepts presented and serve as building blocks for your own experiment with Cactus.

# Tutorial environment

For this tutorial we will be working with a Jupyter based environment that lets you use the Einstein Toolkit without having to first download it. You can log into the system through https://www.etk-unam.ndslabs.org/
[![jupter hub login](fig/login.png)](https://www.etk-unam.ndslabs.org/)

Once logged in, pick "jupyter" from the list of available applications
![application selection screen](fig/05-select-image.png)

Next start the application and wait for it to fully start up
![start application](fig/06-start-image.png)
![application started](fig/07-image-starting.png)

Finally connect to the jupyter notebook
![connect to jupyter](fig/08-image-ready.png)

# Following along
Once jupyter starts up you will be presented with a window showing a list of directories and files. Click on the one named "EinsteinToolkit.ipynb" which is this file.

This notebook contains all the commends you need to enter as executable cells, mostly marked by `%%bash` with the exception of the next one which changes inot the Cactus directory.

In [4]:
%cd Cactus

[Errno 2] No such file or directory: 'Cactus'
/data/rhaas/ncsa/talks/mx-cactusintro/Cactus


which changes into the Cactus directory and outputs its full path. We will be working from inside this directory for now.

Remeber the blue and red stickies to indicate that you are still working and that you need help.

# Obtaining the Einstein Toolkit on your laptop

The Einstein toolkit uses the `GetComponent` tool to download the components.
```bash
curl -O -L https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents
chmod a+x GetComponents
./GetComponents https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2018_02/einsteintoolkit.th
```
GetComponents takes a `thornlist` as its argument which describes how to download the Cactus components
```
!CRL_VERSION = 1.0

# CactusExamples thorns
!TARGET   = $ARR
!TYPE     = git
!URL      = https://bitbucket.org/cactuscode/cactusexamples.git
!REPO_PATH= $2
!CHECKOUT = CactusExamples/HelloWorldCUDA
CactusExamples/HelloWorld
```

# Compiling Cactus
The Einstein toolkit uses a tool `simfactory` to abstract away differences between different high-performance cluster environemnts and between different operating systems. To compile Cactus one uses the `build` comand. We will use it to compile a simple 'Hello, World!' code using Cactus.

In [35]:
%%bash
simfactory/bin/sim build --thornlist thornlists/hello.th hello

bash: line 1: simfactory/bin/sim: No such file or directory


which will work for a while and then end with
```
Done.
```

# Running a first Cactus simulation
`simfactory` also allows to `submit` a simulation to a queueing system, but for this first simulation, and to show what is going on behind the scenes, we will start Cactus directly

In [36]:
%%bash
exe/cactus_hello par/hello.par

bash: line 1: exe/cactus_hello: No such file or directory


This produces quite a bit of output which ends in
> INFO (HelloWorld): Hello Mexico!

Congratulations to (potentially) your first Cactus run!

# Modifying simulation parameters
A cactus simulation takes runtime parameters from a parameter file that is specified on the command line. In the example above the parameter file was `par/hello.par` and looks like so
```
ActiveThorns = "HelloWorld"

HelloWorld::who = "Mexico"
```
which enables the `thorn` (more on this in a second) `HelloWorld` and sets its parameter `who` to the string `"Mexico"`.
Paramters can be strings, integers, floating point numbers or of a set of programmer defined keywords. Cactus verifies that parameters are of the correct type, eg. no floating point numbers are given where integers are expected, and that parameters are within thorn allowed ranges, eg. a parameter specifying a distance may be required to be positive.

If you like, go ahead and change the value to something else, for example your name and re-run the simulation.

In [37]:
%%bash
sed -i 's/Mexico/Roland/' par/hello.par
exe/cactus_hello par/hello.par

sed: can't read par/hello.par: No such file or directory
bash: line 2: exe/cactus_hello: No such file or directory


# Taking a step back

Now that we have seen how to run a Cactus simulation and to modify parameters, we take a step back back to see how this all fits in with Cactus' design principles.

# Cactus concepts

| **term** | **description** |
|------|------|
| **thorn** | Cactus module providing physics or infrastructure capabilities |
| **grid function** | discrete representation of function eg. density $\rho(x, y , z)$ on the numerical grid, eg. $\texttt{rho}_{i,j,k}$ |
| **grid scalar** | single number (or set of numbers) managed by Cactus, eg. total mass |
| **schedule** | order in which Cactus calls user supplied routines, defined through ordering relations `SCHEDULE compute_rho BEFORE compute_mass` |
| **parameter** | user specifyable option controlling simulation. Used for example to provide central density in TOV stars, give final time of simulation, specify type of equation of state |
| **parameter file** | set of parameter values used in a single simulation run, key—value pairs |

# Cactus directory layout

<pre class='floatright'>
arrangements
 └CactusExamples
    └HelloWorld
configs
  ├hello
  └mclachlan
exe
  ├cactus_hello
  └cactus_mclachlan
par
  ├hello.par
  └qc0.par
simfactory
src
thornlists
</pre>
  - Cactus lets you arrange its modules (`thorns`) in `arrangements` to group together similar modules. For example the HelloWorld `thorn` is in the `CactusExamples` arrangement
  - Compiled executables are in `exe` while all object files etc. are in the subdirectories of `configs` which are the `configurations` that have been build. Each `configuration` can use a different subset of `thorns` or different compile options.
  - `src` is `Cactus`'s own source code, you usally do not have to look into it
  - Often parameter files, will be stored in a directory `par`. This is optional and you can store them wherever you like.

# Thorn directory layout

<pre class='floatright'>
HelloWorld
  ├README
  ├interface.ccl
  ├param.ccl
  ├schedule.ccl
  ├doc
  │  └documentation.tex
  ├par
  │  └HelloWorld.par
  └src
     ├HelloWorld.c
     └make.code.defn
</pre>
  - the three files ending in `.ccl` are `C`actus `c`configuration fi`l`es, that describe:
       - the `interface` that a thorn implements (think `java`)
       - the `param`eters that a thorn understands
       - the `schedule` of its functions during the simulation
  - `make.code.defn` lists all source files that need to be compiled to build the thorn
  - and finally `HelloWorld.c` is the actual source code

`param.ccl` defines the parameter `who` that we had used before:

In [52]:
%%bash
cat arrangements/CactusExamples/HelloWorld/param.ccl

# Parameter definitions for thorn HelloWorld
# $Header$


this declares `who` to be a string, which must not be empty (since only values that match the regular expression `".+"` are allowed) and whose default values is `"World"`. We also have to provide a descriptive text for the parameter itself `"who to greet"` and of each allowed range of values `"we accept any non-emtpy string"`.

`interface.ccl` defines the variables that the `thorn` provides and which `thorns` it `inherit`s from:

In [51]:
%%bash
cat arrangements/CactusExamples/HelloWorld/interface.ccl

# Interface definition for thorn HelloWorld
# $Header$

implements: helloworld

which is almost empty since `HelloWorld` does not define any variables or inherit from any other thorn.

`schedule.ccl` determines which functions are executed in which order and also what variables defined in `interface.ccl` will be provided with storage (there storage being *scheduled*):

In [53]:
%%bash
cat arrangements/CactusExamples/HelloWorld/schedule.ccl

# Schedule definitions for thorn HelloWorld
# $Header$

schedule HelloWorld at CCTK_EVOL
{
  LANG: C
} "Print message to screen"

which schedules the function `HelloWorld` in the `CCTK_EVOL` timebin. `HelloWorld` is written in the `C` (or `C++`) language and is describe to "Print message to screen".

# Binary black hole merger simulations

The Einstein Toolkit provides all tools required to simulate the merger of two black holes and to analyse the resulting gravitational waves. 

  * a thorn to compute initial data for black holes: `TwoPunctures`
  * a spacetime evolution code: `McLachlan`
  * codes to compute the gravitational waves produced by the system `WeylScal4` and `Multipole`
  
Ian Hinder provides a set of parameter files that simulate LIGO's first detected black hole merger event http://einsteintoolkit.org/gallery/bbh/index.html
![gravitational waves emitted by a black hole merger event](fig/Strain_sm.png)

# Binary black hole toy merger

The LIGO detection is a too large to simulate on the tutorial machines, so we use a toy-version of it that only includes the last 1/2 orbit of the merger only.

In [55]:
%%bash
simfactory/bin/sim submit --configuration qc0 --walltime 1:00:00 --procs 4 --num-threads 2 --parfile par/qc0-mclachlan-small.par qc0

bash: line 1: simfactory/bin/sim: No such file or directory


This does something new. It uses simfacory to `submit` a simulation, ie it runs it in the background, freeing up the terminal.
  - it uses a configuration in `configs` called qc0
  - it asks for the simulation to run for at most 1:00:00 hours
  - it uses 4 cores, which happens to be the number of cores allocated to each tutorial machine
  - uses 2 threads per MPI rank (so 4/2 = 2 MPI ranks)
  - uses the paramter file parqwc0.par
  - names the simulation qc0

This will finish in about XXX minutes. 

# Simulation output

Simulation output is written to a cluster specific location. You can query the location using simfactory

# Lunch ...